<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Agentic AI Workflows - Week 2</center></font>

<center><p float="center">
  <img src="https://images.pexels.com/photos/33910465/pexels-photo-33910465.jpeg" width=720></a>
<center><font size=6>Greatglobe Logistics Assistant: Automating Product Compliance and Customs Guidance</center></font>

#**Problem Statement**

## Business Context

Greatglobe Logistics is a global supply chain company that manages the shipment of more than 500 products for 20-30 client companies across multiple domains. Each shipment may require compliance with international trade regulations, including import/export documentation, customs duties, and payment procedures. For logistics managers and supply chain teams, gathering accurate documentation and understanding payment obligations for each shipment is time-consuming and prone to errors. Automating this process using data and intelligent agents can greatly improve operational efficiency.

Currently, retrieving product details and corresponding trade compliance requirements is a manual, fragmented process:

- Product information is stored in internal databases (Product_ID, category, cost, etc.).
- Compliance information such as required import/export documents, duty payment methods, and obligations must be researched online, which is slow and inconsistent.

This makes it difficult for logistics teams to quickly verify shipments, especially when handling multiple clients across different countries.

##  Objective

The goal of this project is to build an interactive logistics assistant that:

- Retrieves product details from a SQL database based on a user-provided Product_ID.
- Uses a web agent to fetch up-to-date import/export documents and payment methods for the product’s HS/HSN code, based on source and destination countries.
- Presents the information in a structured format, highlighting both required documentation and any additional compliance information if available.

## Data Description

The database contains product-level information that includes:

- Product_ID - Unique identifier for each product.
- Product_Name - Name of the product.
- Category - Broad classification of the product (e.g., Electronics, Textiles, Pharmaceuticals).
- HSN_Code - Harmonized System of Nomenclature code for customs classification.


#**Installing and Importing Necessary Libraries**

In [ ]:
# Install Tavily for AI-optimized web search and LangChain Community for agent tools
# Tavily provides real-time compliance information; LangChain enables SQL and web agents
!pip install -q tavily-python==0.7.12 \
                langchain-community==0.3.29

**Note:**

- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [ ]:
# Import required libraries for database operations, LangChain agents, and web search
# SQL tools for product database queries, Tavily for compliance searches, OpenAI for formatting
import sqlite3                                                       #Imports Python's built-in SQLite library to create and interact with SQLite databases.
import pandas as pd                                                  #Imports Pandas for data manipulation and reading CSV files.
from openai import OpenAI                                            #Imports OpenAI for parsing search results (Optional)
import json                                                          #Imports Json for handling search results
import os
from langchain.agents import create_sql_agent
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import json
from tavily import TavilyClient                                      #Tavily Search API, which performs real-time web searches.

In [ ]:
# Load OpenAI API credentials from local config.json file
# Required for: SQL agent (database queries), web agent (tool orchestration), and response formatting
import json
import os
from openai import OpenAI

# Load the JSON file and extract values
file_name = 'config.json'                                                       # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    API_KEY = config.get("OPENAI_API_KEY")                                      # Extract the API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")                             # Extract the OpenAI base URL from the config

# Store API credentials in environment variables
os.environ['OPENAI_API_KEY'] = API_KEY                                          # Set API key as environment variable
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE                                 # Set API base URL as environment variable

# Initialize OpenAI client
client = OpenAI()                                                               # Create an instance of the OpenAI client

In [ ]:
# Redundant API loading (alternative approach) - can be removed if previous cell works
# Loads same credentials but stores in different variable names
# Load the JSON file and extract values
file_name = 'config.json'
with open(file_name, 'r') as file:
    config = json.load(file)
    OPENAI_API_KEY = config.get("OPENAI_API_KEY") # Loading the API Key
    OPENAI_API_BASE = config.get("OPENAI_API_BASE") # Loading the API Base Url


# Storing API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE

#**SQL Agent**

SQL agents help retrieve the product category based on the given product ID.

In [ ]:
# Initialize SQL agent to query the Greatglobe product database
# Agent translates natural language queries into SQL and retrieves product categories
# Uses gpt-3.5-turbo with temperature=0 for consistent, deterministic database queries
# Initialize the SQLDatabase object
# Use absolute path with triple slashes for sqlite
db = SQLDatabase.from_uri("sqlite:///../data/greatglobe.db")

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # You might need to set your OpenAI API key

# Initialize a SQL agent to interact with the database using the LLM
sql_agent = create_sql_agent(
    llm,
    db=db,
    agent_type="openai-tools",
    verbose=False
)

/var/folders/t1/m6dx844s77gcn0m2bfbk8xmm0000gn/T/ipykernel_40202/4225209325.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # You might need to set your OpenAI API key


In [ ]:
# Test SQL agent with a sample query to retrieve product category for Product ID P1016
# Demonstrates agent's ability to understand natural language and generate appropriate SQL
# Returns the product category needed for compliance searches
query = f"Fetch the category with product_id P1016"


# Execute the query using the SQL agent and store the output
output = sql_agent.invoke(query)

# Display the retrieved records
output['output']

'The category for the product with product_id P1016 is "Pharmaceuticals".'

#**Tavily Client**

**Instruction:**
To create a Tavily API key, please follow these steps:

1. Click the link provided to go to the [Tavily](https://app.tavily.com/home) website.
2. Sign in to the Tavily website.
3. Generate a new API key for your use.
4. Copy the API key and store it securely for use in your project.

In [ ]:
# Initialize Tavily client for AI-optimized web search of compliance information
# Test query searches for import/export documents and payment methods for FMCG USA→India shipment
# Tavily returns structured, LLM-friendly results ideal for customs/trade regulation queries
# Replace with your Tavily API Key
TAVILY_API_KEY = "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"

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Build your query dynamically based on product and route
query = f"List required import/export documents and accepted payment methods for customs clearance (e.g., cash, cheque, online, duty payment portal) for goods of FMCG category shipping from USA to India"

# Call Tavily API
try:
    response = tavily_client.search(query)
    print("=== Tavily Response ===")
    print(response)
except Exception as e:
    print("Error fetching data from Tavily API:", e)


=== Tavily Response ===
{'query': 'List required import/export documents and accepted payment methods for customs clearance (e.g., cash, cheque, online, duty payment portal) for goods of FMCG category shipping from USA to India', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://abhyanshshipping.com/step-by-step-guide-to-customs-clearance-in-india-what-every-importer-should-know/', 'title': 'Custom Clearance in India: Step-by-Step Import Guide for 2025', 'content': '**Importing goods into India for the first time can feel overwhelming.** The customs clearance process involves multiple steps, paperwork, and compliance checks with regulations. In essence, customs clearance ensures that **all duties, taxes, and import regulations are properly followed** before you take delivery of your goods. ## **Step-by-Step Process for Import Customs Clearance in India** Below is a list of **key documents required** for import customs clearance in India, along with

#**Logistic Assistant Agent**

## Search Compliance Info

In [ ]:
# Define tool to search for compliance information using Tavily API
# Takes shipment details (category, source, destination) and searches for:
# - Required import/export documents
# - Accepted payment methods for customs clearance
def search_compliance_info(shipment_details: str):
    """
    Uses Tavily API to search for required import/export documents and payment methods.
    """
    # Build the query for Tavily based on category, source, and destination
    tavily_query = f"required import/export documents and accepted payment methods for customs clearance for goods of following category and shipping source and dest{shipment_details}"
    tavily_response = tavily_client.search(tavily_query)
    return tavily_response["results"]

## Format Compliance Info

In [ ]:
# Define tool to format raw Tavily search results into structured output
# Uses OpenAI to extract and organize compliance information into two sections:
# 1. required_documents: List of import/export documents (no duplicates)
# 2. additional_information: Payment modes, duty rates, restrictions, licenses, etc.
def format_compliance_info(tavily_response: str):

    prompt = f"""
    You are a logistics assistant. Read the response throughly before generating a response. understand which countries are being refered to in the response.
    Extract structured information from the following Tavily API response.

    Requirements:
    1. List all required import/export documents clearly under "required_documents". Please ensure that the documents in the list are not repeated.
    2. If there is other important information (e.g., payment modes, duty rates, obligations, restrictions, licenses, etc.),
       list it under "additional_information".
    3. If no additional information is available, do not include the field.

    Tavily response:
    {tavily_response}
    """

    client = OpenAI()

    # Assuming client (OpenAI client) is initialized elsewhere
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return completion.choices[0].message.content

## Create Agent

In [ ]:
# Create the main logistics agent with two tools: search and format
# Agent uses ReAct pattern to: 1) Search Tavily for compliance info, 2) Format results
# ZERO_SHOT_REACT_DESCRIPTION means agent reasons about which tools to use without examples
# Define tools for the agent
tools = [
    Tool(
        name="Search Compliance Info",
        func=search_compliance_info,
        description="Use this tool to search for import/export compliance information using Tavily.",
    ),

     Tool(
        name="Format Compliance Info",
        func=format_compliance_info,
        description="Use this tool to format the raw Tavily search results into a structured output. Input should be the string output from the 'Search Compliance Info' tool."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

/var/folders/t1/m6dx844s77gcn0m2bfbk8xmm0000gn/T/ipykernel_40202/823676538.py:17: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# Main function orchestrating the complete logistics assistant workflow
# Process: 1) Get user input (source, destination, product_id)
#          2) Query SQL database for product category
#          3) Search web for compliance requirements
#          4) Format and display structured compliance information
def Logistic_Assistant_Agent():
    source = input("Enter Source Country: ")
    destination = input("Enter Destination Country: ")
    product_id = input("Enter Product ID: ")

    sql_query = f"Fetch the category for product id '{product_id}'"
    category_result = sql_agent.invoke(sql_query)
    category = category_result['output']
    category = category.replace("", "")

    # Pass structured input to the agent
    agent_input = f"Below are the details for the query Category: {category},Source: {source},Destination: {destination}"
    response = agent.invoke(agent_input)

    print("Compliance Info:")
    print(response['output'])


#**Sample Test Case**

## Test Case 1

- Origin : USA
- Destinantion : Germany
- Product ID : P1016

In [ ]:
# Execute Test Case 1: USA to Germany shipment for Product P1016
# Tests the complete workflow: database query → category retrieval → compliance search → formatting
# Expected inputs when prompted: USA, Germany, P1016
Logistic_Assistant_Agent()

Compliance Info:
Required documents for importing pharmaceutical products from the USA to Germany include a Commercial Invoice, Certificate of Analysis (COA), and Import License or Drug Registration Certificate. Additionally, both Germany and the USA have specific export controls and regulations in place for certain items, and compliance with new packaging and labeling regulations is necessary for pharma companies importing products to the USA. Individuals returning to the EU from non-EU countries may not be automatically liable for import VAT under certain conditions.


## Test Case 2

- Origin : United Kingdom
- Destinantion : France
- Product ID : P1001

In [ ]:
# Execute Test Case 2: UK to France shipment for Product P1001
# Tests with different route and product to verify agent's adaptability across scenarios
# Expected inputs when prompted: United Kingdom, France, P1001
Logistic_Assistant_Agent()

Compliance Info:
The compliance information for shipping automobiles from the United Kingdom to France includes the requirement of a Commercial Invoice, Packing List, Certificate of Conformity (CoC), and Customs Clearance Certificate. Import duties and taxes must be paid for obtaining the Customs Clearance Certificate, and the Single Administrative Document (SAD) is required for all goods imported into the European Union (EU) Member States.


# Conclusion

The proposed **Logistics Assistant Agent** offers a step forward in automating compliance checks and streamlining shipment verification for Greatglobe Logistics. By integrating structured product information from the internal SQL database with real-time compliance intelligence from external sources (via Tavily and OpenAI), the system reduces dependency on manual research and fragmented processes.

From a **business impact perspective**, this solution addresses three key challenges:

1. **Operational Efficiency** - Logistics managers and supply chain teams no longer need to manually search for country-specific compliance documents or duty payment methods. The assistant delivers structured, accurate, and timely information, accelerating decision-making and reducing turnaround time per shipment.

2. **Error Reduction and Compliance Assurance** - Trade compliance errors can be costly, leading to shipment delays, penalties, or reputational risks. Automating document retrieval and payment obligation identification minimizes human oversight and ensures that shipments meet international regulatory requirements consistently.

3. **Scalability Across Clients and Regions** - With Greatglobe handling 20–30 clients and over 500 products across multiple domains, the assistant is designed to scale. It adapts to different product categories and trade routes, making it a versatile tool that supports growth without proportional increases in compliance workload.

Overall, the solution enables efficient management of multiple products and clients across countries, streamlining global trade operations.


<font size = 6 color="#4682B4"><b> Power Ahead </font>
___